# Real search engine

* Upgrade from our toy example to something real
* Test an industry-grade search engine
* Useful skill to have:
  * Work with own corpora - powerful search for free
  * Custom search and indexing
  * Very often needed in the data science industry (surprisingly many projects boil down to setting up custom search)
* The purpose of this lecture is to give a *very* quick idea on how to set up custom search engine
* There is no time to go to any detail, but it is meant to inspire

# Which one?

* Apache Solr
* Elasticsearch
* Vespa.ai
* These are the main contestants on the playground
* We will go with Apache Solr for no very strong reason, the main principles apply to all of them
* These are engines (the backend) not interfaces (frontend)
  
## Solr

* Apache project
* Search **engine** - not so much the interface
  1. Index data
  2. Query via HTTP API
  3. Collect results in various formats
  4. Use them any way you need

# Installing Solr

* For what it is, installing Solr is surprisingly easy
* Google "solr download" and download the latest version (a single .tgz which you unpack)
* Once unpacked, you can use it as follows:

In [ ]:
# Tells solr to start and listen on port 8983 (the default)
bin/solr start 

# Tells solr to stop
bin/solr stop

* After start, you can go to http://127.0.0.1:8983

# Practical problem

* Solr is quite impossible to run in the Colab environment
* You can run it locally
* Watch the video of the lecture to see how

# Solr cores

* Data stored in *cores*
* One core - one dataset
* (collections of cores not covered here)

# Creating a core 

* Cores are like projects for Solr
* One instance of Solr can have multiple cores running simultaneously
* Each core has its own data


In [ ]:
bin/solr create_core -c mytest

http://127.0.0.1:8983/solr/#/mytest

# Adding data

* bin/post -c mytest /dir/with/data
* .... wait for a while, kill, and commit
* curl http://127.0.0.1:8983/solr/mytest/update?commit=true
* ...and enjoy the result...

# Adding data programmatically

* Let's try to index Wiki Quotes
* Do so programmatically to have full control
* And use that to learn a bit about the query language


# Indexing data with Solr

* Using the data we extracted from wikidumps
* Each document will have a title and content
* Indexing principles:
  * Each document is one python dictionary, field names are keys
  * ...

In [ ]:
# Same as in the basic IR lecture
!wget -O fiwiki.xml.bz2 https://www.dropbox.com/s/r82qnfdj1encx1z/fiwiki-20181001-corpus.xml.bz2?dl=1
!wget -nc -O xml2txt.pl https://www.dropbox.com/s/p3ta9spzfviovk0/xml2txt.pl?dl=1
!bzcat fiwiki.xml.bz2 | perl xml2txt.pl -articles /dev/stdin /dev/stdout  | head -n 100000 | tqdm | gzip > fiwiki.txt.gz


In [4]:
! zcat fiwiki.txt.gz | head -n 40

<article name="Amsterdam">
Amsterdam on Alankomaiden pääkaupunki. Amsterdam on väkiluvultaan Alankomaiden suurin kaupunki, huhtikuun alussa 2006 siellä asui 743 905 asukasta eli noin joka 20. hollantilainen asuu Amsterdamissa. Yhteensä Amsterdamissa ja sitä ympäröivällä kaupunkialueella asuu noin 1 450 000 ihmistä eli vajaa kymmenesosa Alankomaiden asukkaista. Amsterdam sijaitsee Amstelin suistossa IJsselmeerin rannalla Alankomaiden Pohjois-Hollannin provinssissa. Vaikka Amsterdam on Alankomaiden perustuslain mukaan maan pääkaupunki, sijaitsevat niin kuningashuone, hallitus, parlamentti kuin korkein oikeuskin Haagissa.Maantiede ja ilmasto
Amsterdam sijaitsee tasaisella alankoalueella, ja osa siitä on merenpinnan tason alapuolella. Kaupunki on IJsselmeeriin kuuluvan IJ’n etelärannalla. Amsteljoki virtaa kaupungin läpi. Vauraus johti myös taiteen ja luonnontieteen kehitykseen. Rembrandt, Frans Hals, Vermeer ja Paulus Potter vaikuttivat siihen aikaan, ja heidän työnsä olivat erittäin kysy

In [5]:
import gzip
def articles(gzipfile,max_articles=1000):
    """A function to yield documents, one at a time"""
    with gzip.open(gzipfile,"rt") as f:
        article=[]
        for line in f:
            line=line.strip()
            article.append(line)
            if line=="</article>":
                yield " ".join(article)
                max_articles-=1
                if max_articles==0:
                    break
                article=[]

for x in articles("fiwiki.txt.gz"):
    print(x)
    break

<article name="Amsterdam"> Amsterdam on Alankomaiden pääkaupunki. Amsterdam on väkiluvultaan Alankomaiden suurin kaupunki, huhtikuun alussa 2006 siellä asui 743 905 asukasta eli noin joka 20. hollantilainen asuu Amsterdamissa. Yhteensä Amsterdamissa ja sitä ympäröivällä kaupunkialueella asuu noin 1 450 000 ihmistä eli vajaa kymmenesosa Alankomaiden asukkaista. Amsterdam sijaitsee Amstelin suistossa IJsselmeerin rannalla Alankomaiden Pohjois-Hollannin provinssissa. Vaikka Amsterdam on Alankomaiden perustuslain mukaan maan pääkaupunki, sijaitsevat niin kuningashuone, hallitus, parlamentti kuin korkein oikeuskin Haagissa.Maantiede ja ilmasto Amsterdam sijaitsee tasaisella alankoalueella, ja osa siitä on merenpinnan tason alapuolella. Kaupunki on IJsselmeeriin kuuluvan IJ’n etelärannalla. Amsteljoki virtaa kaupungin läpi. Vauraus johti myös taiteen ja luonnontieteen kehitykseen. Rembrandt, Frans Hals, Vermeer ja Paulus Potter vaikuttivat siihen aikaan, ja heidän työnsä olivat erittäin kysy

# Import data to Solr

* This can be done in many ways
* The pysolr library is easy
* Remember to pip install it

In [8]:
!pip install pysolr

In [10]:
import pysolr

# Connecting to solr
solr = pysolr.Solr("http://localhost:8983/solr/mytest") #<--- note the core name here

for docid,wikipage in enumerate(articles("fiwiki.txt.gz",2000)):
    document_dict={"id":docid,"article_txt_fi":wikipage}
    solr.add(document_dict) #this would be **much** faster if you add a list of documents at once
solr.commit() #commit every now and then, no need to commit after every add()

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">890</int>\n</lst>\n</response>\n'

# Why did we name the field article_txt_fi

* Anything ending with \_txt_fi is understood by Solr as "Finnish text" in the default configuration
* These so called "dynamic fields" make testing and quick prototyping a breeze
* The field's definition can be inspected under Schema / Field / Load Term Info
  * There we see the exact processing done at indexing time and at query time


# Query

* In the default configuration, there is no "nice" google-style user interface (unfortunately)
* The admin interface's query function will have to do for now
* There are different query parsers in Solr, but let us try the default one:
  * df (default field) is article_txt_fi
  * select "hl" (highlight)
  * "hl.fl" is article_txt_fi
  * the query itself goes under "q"

# Just a handful of the many query types you can post

* Amsterdam
* Amsterdam kaupunki (same as Amsterdam OR kaupunki)
* Am*dam
* Amsterdam?
* Amstrdam~0.8
* "Amsterdam siirtokunta"
* +Amsterdam -Hollanti -Alankomaat
* "Amsterdam Rotterdam"~5


https://solr.apache.org/guide/8_8/the-standard-query-parser.html


In [29]:
# Search is as easy as indexing

params={"hl":"true", "hl.fl":"article_txt_fi","df":"article_txt_fi"}
results=solr.search("Amsterdam kaupunki",**params)
#dir(results)
for d in results.docs:
    print(d)
    break
print(results.highlighting)
    


{'id': 'efa4045b-af63-482a-a424-3120e14e7a56', 'article_txt_fi': '<article name="Amsterdam"> Amsterdam on Alankomaiden pääkaupunki. Amsterdam on väkiluvultaan Alankomaiden suurin kaupunki, huhtikuun alussa 2006 siellä asui 743 905 asukasta eli noin joka 20. hollantilainen asuu Amsterdamissa. Yhteensä Amsterdamissa ja sitä ympäröivällä kaupunkialueella asuu noin 1 450 000 ihmistä eli vajaa kymmenesosa Alankomaiden asukkaista. Amsterdam sijaitsee Amstelin suistossa IJsselmeerin rannalla Alankomaiden Pohjois-Hollannin provinssissa. Vaikka Amsterdam on Alankomaiden perustuslain mukaan maan pääkaupunki, sijaitsevat niin kuningashuone, hallitus, parlamentti kuin korkein oikeuskin Haagissa.Maantiede ja ilmasto Amsterdam sijaitsee tasaisella alankoalueella, ja osa siitä on merenpinnan tason alapuolella. Kaupunki on IJsselmeeriin kuuluvan IJ’n etelärannalla. Amsteljoki virtaa kaupungin läpi. Vauraus johti myös taiteen ja luonnontieteen kehitykseen. Rembrandt, Frans Hals, Vermeer ja Paulus Potte